# Ollama PDF RAG Notebook

In [ ]:
!apt-get install poppler-utils
!pip install \
    langchain==0.3.14 \
    langchain-core==0.3.29 \
    langchain-ollama==0.2.2 \
    langchain_community==0.3.14 \
    langchain_text_splitters==0.3.5 \
    unstructured[all-docs] \
    pdfminer.six \
    pi_heif \
    unstructured_inference \
    pdf2image \
    chromadb \
    pydantic==2.9.2 \
    gradio -q

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 2s (117 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 66.7 MB/s eta 0:00:00
  Preparing metadata (s

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries


In [ ]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.caches import BaseCache
from langchain.cache import InMemoryCache # Import a concrete implementation of BaseCache

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [ ]:


# List all your PDF paths here
pdf_paths = [
    "/content/drive/MyDrive/DGMS_Data/ma_1952.pdf",
    "/content/drive/MyDrive/DGMS_Data/mmr.pdf",
    "/content/drive/MyDrive/DGMS_Data/Coal Mines Regulation 2017.pdf",
    # add more paths as needed
]

docs = []
for path in pdf_paths:
    if path:
        loader = UnstructuredPDFLoader(file_path=path)
        pdf_docs = loader.load()
        # Tag source so you can filter or inspect later if needed
        for d in pdf_docs:
            d.metadata["source_file"] = path
        docs.extend(pdf_docs)
        print(f"PDF loaded successfully: {path}")
    else:
        print("Skipped an empty path.")

print(f"Total documents loaded: {len(docs)}")


PDF loaded successfully: /content/drive/MyDrive/DGMS_Data/ma_1952.pdf
PDF loaded successfully: /content/drive/MyDrive/DGMS_Data/mmr.pdf
PDF loaded successfully: /content/drive/MyDrive/DGMS_Data/Coal Mines Regulation 2017.pdf
Total documents loaded: 3


## Split text into chunks

In [ ]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)
print(f"Text split into {len(chunks)} chunks")

Text split into 1290 chunks


In [ ]:
import subprocess
import os

!curl -fsSL https://ollama.com/install.sh | sh


# Start Ollama in the background
serve_proc = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


# Wait for Ollama to start (give it a few seconds)
import time
time.sleep(5)


# Download the models you want to use
!ollama pull nomic-embed-text
!ollama pull llama3.2
print("Ollama server and models are running.")

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Ollama server and models are running.


## Create vector database

In [ ]:
# Create vector database
embedding = OllamaEmbeddings(model="nomic-embed-text")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    collection_name="local-rag",
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [ ]:
# Set up LLM and retrieval
local_model = "llama3.2"
llm = ChatOllama(model=local_model)
llm.model_rebuild()

In [ ]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant.
    Use the following DGMS guidelines to answer the question and generate several focused search queries. If you don't know the answer based on the context, say so. Always cite the relevant section. gen
    Original question: {question}""",
)


# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [ ]:
# RAG prompt template

template = """
You are a mining safety and DGMS compliance expert.
Use ONLY the DGMS context below to answer.

Context:
{context}

Task:
1. First, classify the user question:
   - If it is asking for a definition, meaning, scope, objectives, or main idea of an Act/Rule/Regulation (for example: 'define', 'what is', 'explain', 'main idea'), treat it as a DEFINITION question.
   - If it describes an incident, accident, unsafe condition, non-compliance, or explicitly asks for 'cause', 'why', 'root cause', 'analysis', or 'recommendations', treat it as an INCIDENT / ROOT CAUSE question.
   - If it mixes both, answer both, but keep root cause and recommendations strictly grounded in the context.

2. For DEFINITION questions (only definition asked):
   - First, check whether the provided DGMS / Mines Act context explicitly defines the term.
     - If a definition is present (for example, in a definitions section like Section 2 of the Mines Act, 1952), quote that meaning in simple language and clearly mention the section number and clause (e.g. “Section 2(b) of the Mines Act, 1952 defines ‘adult’ as a person who has completed his eighteenth year.”).
     - Do NOT replace this statutory definition with a generic or approximate wording.
   - Only if NO explicit definition or clause for the term is present in the context:
     - State clearly: “The provided DGMS/Mines Act context does not contain an explicit definition of ‘<term>’.”
     - Then you may add a short, generic explanation like “In general usage, an adult is a person who is 18 years of age or above”, but mark it as “general understanding, not a formal definition from the provided text”.
   - Never say that “no specific section defines <term>” if the context actually includes such a definition.
   - Do NOT fabricate or assume incidents for pure definition questions.
   - Do NOT perform root cause analysis or give recommendations unless the user has explicitly asked for causes or preventive measures.


3. For INCIDENT / ROOT CAUSE questions:
   - DGMS Explanation:
     - Identify and briefly explain the relevant DGMS provisions from the context (with section or rule identifiers where available).
   - Root Cause Analysis:
     - Problem summary: Restate the incident / non-compliance in 2–3 sentences, using only what is in the question and context, without assuming extra details.
     - Immediate causes: List the most likely direct causes supported by the context.
     - Underlying causes: List deeper systemic causes (for example: organisation, procedure, training, supervision, maintenance, work environment) supported by the context.
     - If the context does not specify a cause, clearly say "Not specified in the provided DGMS context" instead of guessing.
   - Recommendations:
     - ONLY provide recommendations if the user explicitly asks for "recommendation", "prevent", "corrective action", or "what should be done".
     - When asked, give 3–5 specific, practical recommendations linked to relevant DGMS provisions from the context.
     - If some recommendation is common-sense but not in the context, mark it as "general good practice (not explicitly stated in the provided DGMS text)".

4. General constraints:
   - Never invent Acts, section numbers, or accident scenarios that are not present in the context.
   - If the Mines Act in the context is different from what the user seems to assume, explain only what is actually present in the context.
   - If the answer cannot be derived from the context, clearly say so.

Format the final answer as:

DGMS Explanation
- ...

Root Cause Analysis (only if it is an INCIDENT / ROOT CAUSE question)
- Problem summary: ...
- Immediate causes: ...
- Underlying causes: ...

Recommendations (only if the user has asked for recommendations / corrective actions)
- ...

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [ ]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [ ]:
# Example 1
chat_with_pdf("What is the main idea of Mines Act?")

DGMS Explanation
- The Mines Act, 1952 is an Act to amend and consolidate the law relating to the Regulation of labour and safety in mines.

Recommendations 
- Ensure that all necessary measures are taken to eliminate or minimize risks to safety and health for persons employed in mines under control.
- Conduct regular inspections and maintain records to ensure compliance with relevant provisions of the Mines Act, 1952.
- Provide adequate training and supervision to employees working in mines.

In [ ]:
# Example 2
chat_with_pdf("Define adult according to mines act?")

Section 2(b) of the Mines Act, 1952 defines ‘adult’ as a person who has completed his eighteenth year.

In [ ]:
# Example 3
chat_with_pdf("Describe regulations for night shifts?")

DGMS Explanation
- Night shifts are governed by regulation 14(2)(b) of the Mines Act, 1952. This states that no work shall be carried on below ground in any mine except by a system of Shifts so arranged that the period of work for each shift is not spread-over more than the daily maximum hours stipulated.
- The standard for night shifts as per the regulation is that if a person employed in a mine works on a shift which extends beyond midnight, then a weekly day of rest shall mean a period of twenty-four consecutive hours beginning when his shift ends (regulation 14(2)(a)).

Recommendations
- Ensure adequate lighting during all periods of work, including night shifts.
- Implement and enforce the daily maximum hours for night shifts as per regulation 14(2) to prevent overwork and ensure rest time is provided.
- Provide adequate supervision and control during night shifts to secure safe operation.

In [ ]:
import gradio as gr

def respond_to_query(message, history):
    if not message.strip():
        return "Please enter a valid question."
    try:
        response = chain.invoke({"question": message})
        return response if response and response.strip() else "No answer found for your query."
    except Exception as e:
        return f"Error processing query: {str(e)}"


# Custom theme tweaking textbox and button colors only via theme API
custom_theme = gr.themes.Soft(
    primary_hue="indigo",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("Verdana"), "Verdana", "sans-serif"],
).set(
    # Textbox background and border color
    body_background_fill="#ffffff",
    input_background_fill="#f7f9ff",
    input_border_color="#ffffff",
    # Button colors
    button_secondary_background_fill="#ffffff",
    button_secondary_background_fill_hover="#f7f9ff",
    button_secondary_border_color="#ffffff",
    button_secondary_text_color="#ffffff",
)

with gr.Blocks(theme=custom_theme) as demo:
    gr.Markdown("# 📄 Mining Safety Q&A")

    chatbot = gr.Chatbot(
        height=500,
        label="Chat History",
        show_copy_button=True
    )

    with gr.Row():
        msg = gr.Textbox(
            label="Your Question",
            placeholder="Ask anything..",
            scale=4,
        )
        submit_btn = gr.Button(
            "Send",
            scale=1,
            variant="secondary",  # lighter, bordered look
        )

    with gr.Row():
        clear_btn = gr.Button(
            "Clear Chat",
            variant="secondary",
            size="sm",
        )

    gr.Examples(
        examples=[
            "Write in brief about Mines Act?",
            "What are the safety regulations in CMR?",
            "What are the ventilation requirements in MMR?",
            "Date - 04.09.16 Mine - RAVINDRA KHANI NO.5 Time - 1.45 Owner - Singareni Collieries Company Ltd. Dist. - Adilabad, State - Telangana Person(s) Killed :  1. Badri Janardhan,Supportman, Male, 54YearsWhile a group of five persons were erecting cog in an underground depillaring panelof a coal mine after withdrawing supports from goaf edge, all of a sudden a mass ofresin bolted coarse grained sandstone roof measuring about 13.2m (length) x 1.6m to4.4m (width) x 0.35m to 1.5m(height) fell from height of about 1.60m on twopersons, completely burying them; one person died instantaneously, another rescuedwith serious injuries after about 4 hrs and three persons escaped unhurt. The area had shown signs of deteriorating ground conditions earlier in the shift, including audible popping sounds and small rock fragments falling from the roof. The on-shift supervisor had been informed but allowed work to continue, stating the section needed to stay on schedule. No additional support, such as temporary props or supplemental bolts, was installed prior to the failure.?"
        ],
        inputs=msg
    )

    def user_message(user_input, chat_history):
        if not user_input.strip():
            return "", chat_history
        return "", chat_history + [[user_input, None]]

    def bot_response(chat_history):
        user_input = chat_history[-1][0]
        bot_reply = respond_to_query(user_input, chat_history)
        chat_history[-1][1] = bot_reply
        return chat_history

    msg.submit(user_message, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_response, chatbot, chatbot
    )
    submit_btn.click(user_message, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot_response, chatbot, chatbot
    )
    clear_btn.click(lambda: [], None, chatbot, queue=False)

demo.launch(share=True, debug=True, max_threads=10)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8a1528c7fa9e2ccc48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8a1528c7fa9e2ccc48.gradio.live


## Clean up (optional)

In [ ]:
# Optional: Clean up when done
vector_db.delete_collection()
print("Vector database deleted successfully")